<a href="https://colab.research.google.com/github/BRomans/IdMind/blob/main/autoencoder_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os
import pickle
from pathlib import Path
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

In [ ]:
drive.mount("/content/drive")
filepath = "/content/drive/MyDrive/ml2-eeg-biometrics/eeg_dataset_right_hand_task_subset_9channels.csv"

In [ ]:
df=pd.read_csv(filepath)
print(df.shape)
df.dropna(axis=1,how='all',inplace=True)
print(df.shape)

In [ ]:
# Need to exclude S1 0724 Run2 Trial 10 because of incomplete data.
df = df[~ ((df['Participant']=='S1') & (df['Date']==20200724) & (df['Run']=='Run2') & (df['Trial']==10))]
df.reset_index(drop=True, inplace=True) # Reset the index because of the dropped rows.

df.shape

In [ ]:
# Create a train-validation-test split.
# For testing we want to take a separate person entirely (let's take S12) - this constitutes 5% of the total dataset.
# For validation we want to keep it equal across participants and sessions. We'll take 1 run per session for each of the participants. 
# # This is one sixth of the remaining data which equates to 15.8% of the total dataset. Thus, we keep 79.2% for training data.

id_cols = ['Participant','Date', 'Run', 'Task', 'Trial']

# Convert Partipant ID to an integer column 'Target' so that the NN can handle it.
df['Target'] = df['Participant'].astype('category').cat.codes.values

# test_arr = np.array(df[df['Participant']=='S12'])
# print("test_arr shape:", test_arr.shape)

# Run5 looks most complete in the data.
valid_arr = np.array(df[(df['Run']=='Run5') & (df['Participant']!='S12')])
print("valid_arr shape:", valid_arr.shape)

# # Take the rest for training data
# train_arr = np.array(df[~ ((df['Run']=='Run5') & (df['Participant']=='S12'))])
# print("train_arr shape:", train_arr.shape)

In [ ]:
def reshape_data(arr, n_timepoints=2500):
  """ 
  Function to split the ID, target & feature columns and reshape the data into the 
    required format for the AE.
  Input:
      arr   np.ndarray containing the 5 ID columns, the channel measurements and integer target (ID)
  Output: 
    [All np.ndarrays]
      x_arr   The feature values in shape [n_samples, timepoints, features]
      y_arr   The (integer) target values in shape [n_samples, 1, 1]
      id_arr  The categorical identifiers (Date, Run, etc.) in shape [n_samples, 1, 5]
  """
  x_arr = arr[:,5:14] # Exclude the first 5 ID columns and the 15th column (Target)
  y_arr = arr[:,14] # Last column - target (participant ID as integer)
  id_arr = arr[:,:5] # ['Participant','Date', 'Run', 'Task', 'Trial']

  # Find dimensions for new shape.
  n_rows = len(x_arr)
  n_samples = int(n_rows/n_timepoints)
  
  # Reshape the three arrays to the required shape.
  x_arr = x_arr.reshape((n_samples, n_timepoints, x_arr.shape[1]))
  y_arr = y_arr.reshape((n_samples, n_timepoints, 1))
  id_arr = id_arr.reshape((n_samples, n_timepoints, id_arr.shape[1]))

  # We do not need the target values / ID values to be replicated n_timepoints for each sample.
  # Reduce these to just 1 value per sample.
  y_arr_reduced = np.amin(y_arr, axis=1, keepdims=True) # Take the minimum along the second dimension.
  # Check if this is the same as the max value, if not then something has gone wrong.
  if not np.all(y_arr_reduced==np.amax(y_arr, axis=1, keepdims=True)):
    raise ValueError("The target value for each sample does not look to be consistent.")

  id_arr_reduced = np.amin(id_arr, axis=1, keepdims=True) # Take the minimum along the second dimension.

  print("x_arr shape: {} \ny_arr shape: {} \nid_arr shape: {}".format(    \
                                x_arr.shape, y_arr_reduced.shape, id_arr_reduced.shape))

  return x_arr, y_arr_reduced, id_arr_reduced


In [ ]:
b = np.arange(15*6).reshape(6,15)
# Ensure we have the same value in the 15th column for all samples from the same target. (n_timepoints=2)
b[0:3, 14] = 10
b[3:6, 14] = 20
print(b)
reshape_data(b, n_timepoints=3)
# reshape_data(b, n_timepoints=2) throws an error since the target value is not the same for each sample.

In [ ]:
x_valid, y_valid, id_valid = reshape_data(valid_arr)

In [ ]:
# x_test = torch.tensor(test_df.drop('Target', axis=1).values)
# y_test = torch.tensor(test_df['Target'].values)

# x_valid = torch.tensor(valid_df.drop('Target', axis=1).values)
# y_valid = torch.tensor(valid_df['Target'].values)

# x_train = torch.tensor(train_df.drop('Target', axis=1).values)
# y_train = torch.tensor(train_df['Target'].values)

In [ ]:
# df = pd.read_csv('File_explaination.csv')
# print(df)
# files = df.get('Filename')
# print(files[8384])

#Bearing Failure Anomaly Detection
In this workbook, we use an autoencoder neural network to identify vibrational anomalies from sensor readings in a set of bearings. The goal is to be able to predict future bearing failures before they happen. The vibrational sensor readings are from the NASA Acoustics and Vibration Database. Each data set consists of individual files that are 1-second vibration signal snapshots recorded at 10 minute intervals. Each file contains 20,480 sensor data points that were obtained by reading the bearing sensors at a sampling rate of 20 kHz.

This autoencoder neural network model is created using Long Short-Term Memory (LSTM) recurrent neural network (RNN) cells within the Keras / TensorFlow framework.

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed
from tensorflow import set_random_seed
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

In [ ]:
# set random seed
seed(10)
set_random_seed(10)

#Data loading and pre-processing
An assumption is that mechanical degradation in the bearings occurs gradually over time; therefore, we use one datapoint every 10 minutes in the analysis. Each 10 minute datapoint is aggregated by using the mean absolute value of the vibration recordings over the 20,480 datapoints in each file. We then merge together everything in a single dataframe.

In [ ]:
data_dir = 'data/bearing_data'
merged_data = pd.DataFrame()

for filename in os.listdir(data_dir):
    dataset = pd.read_csv(os.path.join(data_dir, filename), sep='\t')
    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1,4))
    dataset_mean_abs.index = [filename]
    merged_data = merged_data.append(dataset_mean_abs)
    
merged_data.columns = ['Bearing 1', 'Bearing 2', 'Bearing 3', 'Bearing 4']

In [ ]:
# transform data file index to datetime and sort in chronological order
merged_data.index = pd.to_datetime(merged_data.index, format='%Y.%m.%d.%H.%M.%S')
merged_data = merged_data.sort_index()
merged_data.to_csv('Averaged_BearingTest_Dataset.csv')
print("Dataset shape:", merged_data.shape)
merged_data.head()

In [ ]:
train = merged_data['2004-02-12 10:52:39': '2004-02-15 12:52:39']
test = merged_data['2004-02-15 12:52:39':]
print("Training dataset shape:", train.shape)
print("Test dataset shape:", test.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(train['Bearing 1'], label='Bearing 1', color='blue', animated = True, linewidth=1)
ax.plot(train['Bearing 2'], label='Bearing 2', color='red', animated = True, linewidth=1)
ax.plot(train['Bearing 3'], label='Bearing 3', color='green', animated = True, linewidth=1)
ax.plot(train['Bearing 4'], label='Bearing 4', color='black', animated = True, linewidth=1)
plt.legend(loc='lower left')
ax.set_title('Bearing Sensor Training Data', fontsize=16)
plt.show()

In [ ]:
# transforming data from the time domain to the frequency domain using fast Fourier transform
train_fft = np.fft.fft(train)
test_fft = np.fft.fft(test)

In [ ]:
# frequencies of the healthy sensor signal
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(train_fft[:,0].real, label='Bearing 1', color='blue', animated = True, linewidth=1)
ax.plot(train_fft[:,1].imag, label='Bearing 2', color='red', animated = True, linewidth=1)
ax.plot(train_fft[:,2].real, label='Bearing 3', color='green', animated = True, linewidth=1)
ax.plot(train_fft[:,3].real, label='Bearing 4', color='black', animated = True, linewidth=1)
plt.legend(loc='lower left')
ax.set_title('Bearing Sensor Training Frequency Data', fontsize=16)
plt.show()

In [ ]:
# frequencies of the degrading sensor signal
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(test_fft[:,0].real, label='Bearing 1', color='blue', animated = True, linewidth=1)
ax.plot(test_fft[:,1].imag, label='Bearing 2', color='red', animated = True, linewidth=1)
ax.plot(test_fft[:,2].real, label='Bearing 3', color='green', animated = True, linewidth=1)
ax.plot(test_fft[:,3].real, label='Bearing 4', color='black', animated = True, linewidth=1)
plt.legend(loc='lower left')
ax.set_title('Bearing Sensor Test Frequency Data', fontsize=16)
plt.show()

In [ ]:
# normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.transform(test)
scaler_filename = "scaler_data"
joblib.dump(scaler, scaler_filename)

In [ ]:
# reshape inputs for LSTM [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
print("Training data shape:", X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print("Test data shape:", X_test.shape)

In [ ]:
a = np.arange(40).reshape((10, 4))
print(a)
b = a.reshape((5,2,4))
b

In [ ]:

# define the autoencoder network model
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:

# create the autoencoder model
model = autoencoder_model(X_train)
model.compile(optimizer='adam', loss='mae')
model.summary()

In [ ]:

# fit the model to the data
nb_epochs = 100
batch_size = 10
history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.05).history

In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()


#Distribution of Loss Function
By plotting the distribution of the calculated loss in the training set, one can use this to identify a suitable threshold value for identifying an anomaly. In doing this, one can make sure that this threshold is set above the “noise level” and that any flagged anomalies should be statistically significant above the background noise.

In [ ]:
# plot the loss distribution of the training set
X_pred = model.predict(X_train)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=train.columns)
X_pred.index = train.index

scored = pd.DataFrame(index=train.index)
Xtrain = X_train.reshape(X_train.shape[0], X_train.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtrain), axis = 1)
plt.figure(figsize=(16,9), dpi=80)
plt.title('Loss Distribution', fontsize=16)
sns.distplot(scored['Loss_mae'], bins = 20, kde= True, color = 'blue');
plt.xlim([0.0,.5])

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(X_test)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=test.columns)
X_pred.index = test.index

scored = pd.DataFrame(index=test.index)
Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored.head()

In [ ]:
# calculate the same metrics for the training set 
# and merge all data in a single dataframe for plotting
X_pred_train = model.predict(X_train)
X_pred_train = X_pred_train.reshape(X_pred_train.shape[0], X_pred_train.shape[2])
X_pred_train = pd.DataFrame(X_pred_train, columns=train.columns)
X_pred_train.index = train.index

scored_train = pd.DataFrame(index=train.index)
scored_train['Loss_mae'] = np.mean(np.abs(X_pred_train-Xtrain), axis = 1)
scored_train['Threshold'] = 0.275
scored_train['Anomaly'] = scored_train['Loss_mae'] > scored_train['Threshold']
scored = pd.concat([scored_train, scored])

In [ ]:
# plot bearing failure time plot
scored.plot(logy=True,  figsize=(16,9), ylim=[1e-2,1e2], color=['blue','red'])

In [ ]:

# save all model information, including weights, in h5 format
model.save("Cloud_model.h5")
print("Model saved")